# Modern Molecular NNs

We have seen two chapters about equivariances in {doc}`data` and {doc}`Equivariant`. We have seen one chapter on dealing with molecules as objects with permutation equivariance {doc}`gnn`. We will combine these ideas and create neural networks that can treat arbitrary molecules with point clouds and permutation equivariance. We already saw SchNet is able to do this by working with an invariant point cloud representation (distance to atoms), but modern networks mix in ideas from {doc}`Equivariant` along with graph neural networks (GNN). This is a highly-active research area, especially for predicting energies, forces, and relaxed structures of molecules.

```{admonition} Audience & Objectives
This chapter assumes you have read {doc}`data`, {doc}`Equivariant`, and {doc}`gnn`. You should be able to

  * Categorize a task (features/labels) by equivariance  
  * Understand body-ordered expansions
  * Differentiate models based on their message passing, message type, and body-ordering  
```

```{warning}
This chapter is in progress
```

## Running This Notebook


Click the &nbsp;<i aria-label="Launch interactive content" class="fas fa-rocket"></i>&nbsp; above to launch this page as an interactive Google Colab. See details below on installing packages.

````{tip} My title
:class: dropdown
To install packages, execute this code in a new cell. 

```
!pip install dmol-book
```

If you find install problems, you can get the latest working versions of packages used in [this book here](https://github.com/whitead/dmol-book/blob/main/package/setup.py)

````

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import tensorflow as tf
import pandas as pd
import rdkit, rdkit.Chem, rdkit.Chem.rdDepictor, rdkit.Chem.Draw
import networkx as nx
import dmol

my_elements = {6: "C", 8: "O", 1: "H"}

In [ ]:
# This cell is for making plots, not part of examples
import rdkit, rdkit.Chem, rdkit.Chem.rdDepictor, rdkit.Chem.Draw
from myst_nb import glue
import networkx as nx
import dmol

# I hate to do this manually, but I cannot get the
# damn molecular fonts to be big enough
import skunk
import matplotlib.pyplot as plt


def _mol2svg(m, size):
    d = rdkit.Chem.Draw.rdMolDraw2D.MolDraw2DSVG(*size)
    d.DrawMolecule(m)
    d.FinishDrawing()
    return d.GetDrawingText()


m1 = rdkit.Chem.MolFromSmiles("C1CCC2CCCCC2C1")
m2 = rdkit.Chem.MolFromSmiles("C1CCC(C1)C2CCCC2")
s1 = _mol2svg(m1, (200, 200))
s2 = _mol2svg(m2, (200, 200))
_, axs = plt.subplots(1, 2, squeeze=True)
axs[0].set_title("decaline")
axs[1].set_title("bicylopentyl")
axs[0].axis("off")
axs[1].axis("off")
skunk.connect(axs[0], "m1")
skunk.connect(axs[1], "m2")
svg = skunk.insert({"m1": s1, "m2": s2})
with open("lwtest.svg", "w") as f:
    f.write(svg)

# Expressiveness

The Equivariant SO(3) ideas from {doc}`Equivariant` will not work on variable sized molecules because the layers are not permutation equivariant. We also know that graph neural networks (GNNs) have permutation equivariance and, with the correct choice of edge features, rotation and translation invariance. So why go beyond GNNs?

One reason is that the standard GNNs cannot distinguish certain types of graphs relevant for chemistry is they cannot distinguish molecules like decaline and bicylopentyl, which indeed have different properties. Look at the {numref}`decaline-bicylopentyl` below and think about the degree and neighbors of the atoms near the mixing of the rings -- you'll see if you try to use message passing the two molecules are identical. This is known as the Wesifeiler-Lehman Test {cite}`weisfeiler1968reduction`.


```{figure} lwtest.svg
---
alt: "decaline and bicyclopentyl structures drawn side-by-side, which visually are different."
name: "decaline-bicylopentyl"
---
Comparison of decaline and bicylopentyl, which have identical output in most GNNs despite being different molecules.
```

These can be distinguished if we also have (and use) their Cartesian coordinates. We cannot distinguish enantiomers with GNNs, except maybe with pre-computed node attributes. Even those start to breakdown when we have helical chirality that is not centered at any one atom.

These are arguments for using Cartesian coordinates in addition to a GNN, but why use equivariant neural networks as opposed to the simpler invariant networks discussed in {doc}`data`? Most molnet research is for **neural potentials**. These are neural networks that predict energy and forces given atom positions and elements. We know that the force on each atom is given by

\begin{equation}
F\left(\vec{r}\right) = -\nabla U\left(\vec{r}\right)
\end{equation}

where $U\left(\vec{x}\right)$ is the rotation invariant potential given all atom positions $\vec{r}$. So if we're predicting a translation, rotation, and permutation invariant potential, why use equivariance? Part of it is performance. Models like SchNet or ANI are invariant and are not as accurate as models like NequiP or TorchMD-NET that have equivariances in their internal layers. Another reason is that there are indeed specific 3D configurations that should have different energies (according to quantum chemistry calculations), but are invariant if treated with pairwise distance alone {cite}`pozdnyakov2022incompleteness`.

## The Elements of Modern Molecular NNs

There has been a flurry of ideas about molents in the last few years, especially with the advances in equivariant neural network layers. Batatia et al.{cite}`batatia2022design` have proposed a categorization of the main elements of molnets (which they call E(3)-equivariant NNs) that I will adopt here. They categorize the decisions to be made into three parts of the architecture: the atomic cluster expansions (ACE), the body-order of the messages, and the architecture of the message passing neural network (MPNN). This categorization might also be viewed within the GNN theory as node features (ACE), message creation and aggregation (body-order), and node update (MPNN details). See {doc}`gnn` for more details on MPNNs.

This is a relatively new categorization and certainly is not necessary to use. Most papers do not use this categorization and it takes some effort to put models into it. The benefit of thinking about models with this abstractions is it helps us differentiate between the very large number of models now being pursued in the literature. There is also a bit of chaos in teasing out what *differentiates* the best models from others. For example, it took a while to discover that the most important features in NequIP were data normalization and how atom embeddings are treated {cite}`batatia2022design`. This categorization is also improving how these models are designed.

### Atom features

Let's start with the general terminology for an atom. Of course, the input to these networks an atom is just a Cartesian coordinate $\vec{r}_i$ and the element $z_i$. As we pass through GNN layers the features will become larger. The atoms are the nodes. The atom features need to be organized a bit differently than previously because some of the features should be invariant with respect to the group --- SO(3) --- and some need to be equivariant. 

### Atomic Cluster Expansions

An ACE is a per-atom tensor. The main idea of ACE is to encode the local environment of an atom into a feature tensor that describes its neighborhood of nearby atoms. This is like distinguishing between an oxygen in an alcohol group vs an oxygen in an ether. Both are oxygens, but we expect them to behave differently. ACE is the same idea, but for nearby atoms in space instead of just on the molecular graph.

The general equation for ACE (assuming O(3) equivariance) is [cite]:

\begin{equation}
A^{(t)}_{i, kl_3m_3} = \sum_{l_1m_1,l2_m2}C_{l1m_1,l_2m_2}^{l_3,m_3}\sum_{j \in \mathcal{N}(i)} R^{(t)}_{kl_1l_2l_3}\left(r_{ji}\right)Y_{l1}^{m_1}\left(\hat{\mathbf{r}}_{ji}\right)\mathcal{W}^{(t)}_{kl_2}h_{j,l_2m_2}^{(t)}
\end{equation}

Wow! What an expression. Let's go through this carefully, starting with the output. $A^{(t)}_{i, kl_3m_3}$ are the feature tensor values for atom $i$ at layer $t$. There are channels indexed by $k$ and the spherical harmonic indexes $l_3m_3$. The right-hand side is nearly identical to the G-equivariant neural network layer equation from {doc}`Equivariant`. We have the input 

How is this different than a MPNN

```{glue:figure} dframe
----
name: dframe
----
Intermediate step of typical modern molecular neural network. The atom features are encoded as spherical harmonics/fragments and they are grow in dimension as we increase the number of them. The network updates just like a message passing graph neural network, except that we only do mixing among the same fragments following the SO(3) equivariant network approach described in {doc}`Equivariant`. There often is a Clebsch-Gordon non-linearity on the aggregated messages to mix between channels, although that is not shown here.
```


To help understand the GCN layer, look at {numref}`dframe`. It shows an intermediate step of the GCN layer. Each node feature is represented here as a one-hot encoded vector at input. The animation in {numref}`gcnanim` shows the averaging



```{figure} ../_static/images/molnet.gif
----
name: gcnanim
alt: animation of the graph convolution layer operation
----
Animation of the graph convolution layer operation. The left is input, right is output node features. Note that two layers are shown (see title change). As the animation plays out, you can see how the information about the atoms propagates through the molecule via the averaging over neighbors. Note how each fragment set is updated independently to maintain equivariance. 
```

## Normalization

* Physics-based energy/force normalization
* Pooling
* Layers

In [ ]:
# THIS CELL IS USED TO GENERATE A FIGURE
# AND NOT RELATED TO CHAPTER
# YOU CAN SKIP IT


def mol2coords(mol):
    drawer = rdkit.Chem.Draw.rdMolDraw2D.MolDraw2DCairo(-1, -1)
    drawer.drawOptions().scalingFactor = 1.0  # 1 pixel per angstrom
    drawer.DrawMolecule(mol)
    drawer.FinishDrawing()
    return {
        i: [drawer.GetDrawCoords(i).x, drawer.GetDrawCoords(i).y]
        for i in range(mol.GetNumAtoms())
    }


def smiles2graph(sml):
    """Argument for the RD2NX function should be a valid SMILES sequence
    returns: the graph
    """
    m = rdkit.Chem.MolFromSmiles(sml)
    m = rdkit.Chem.AddHs(m)
    order_string = {
        rdkit.Chem.rdchem.BondType.SINGLE: 1,
        rdkit.Chem.rdchem.BondType.DOUBLE: 2,
        rdkit.Chem.rdchem.BondType.TRIPLE: 3,
        rdkit.Chem.rdchem.BondType.AROMATIC: 4,
    }
    N = len(list(m.GetAtoms()))
    nodes = np.zeros((N, len(my_elements)))
    lookup = list(my_elements.keys())
    for i in m.GetAtoms():
        nodes[i.GetIdx(), lookup.index(i.GetAtomicNum())] = 1

    adj = np.zeros((N, N, 5))
    for j in m.GetBonds():
        u = min(j.GetBeginAtomIdx(), j.GetEndAtomIdx())
        v = max(j.GetBeginAtomIdx(), j.GetEndAtomIdx())
        order = j.GetBondType()
        if order in order_string:
            order = order_string[order]
        else:
            raise Warning("Ignoring bond order" + order)
        adj[u, v, order] = 1
        adj[v, u, order] = 1
    pos = mol2coords(m)
    aug_adj = np.zeros((N, N))
    for i in range(adj.shape[0]):
        for j in range(adj.shape[0]):
            if i != j and np.linalg.norm(np.array(pos[i]) - np.array(pos[j])) < 2:
                aug_adj[i, j] = 1
                aug_adj[j, i] = 1
    aug_adj[np.diag_indices(N)] = 0
    return nodes, adj, aug_adj, pos

In [ ]:
# THIS CELL IS USED TO GENERATE A FIGURE
# AND NOT RELATED TO CHAPTER
# YOU CAN SKIP IT
from myst_nb import glue
from moviepy.editor import VideoClip
from moviepy.video.io.bindings import mplfig_to_npimage


def draw_vector(x, y, s, v, color_i, draw_i, ax, cmap, **kwargs):
    x -= s / 2
    # iterate vertically
    L = max([len(vi) for vi in v])
    y += L * s - s * 2 / 4
    for dy, vi in enumerate(v):
        ax.text(
            x,
            y - s / 2 - dy * s,
            "l={}".format(dy),
            verticalalignment="center",
            horizontalalignment="right",
            fontsize=7,
        )
        if dy == 0:
            for dx, vij in enumerate(v[-1]):
                ax.text(
                    x + dx * s * 1.5 / L + s * 1.5 / L / 2,
                    y - dy * s,
                    "k={}".format(dx),
                    verticalalignment="bottom",
                    horizontalalignment="center",
                    fontsize=7,
                )
        for dx, vij in enumerate(vi):
            if cmap is not None and (color_i is None or dy in color_i):
                ax.add_patch(
                    mpl.patches.Rectangle(
                        (x + dx * s * 1.5 / L, y - dy * s),
                        s * 1.5 / L,
                        -s,
                        facecolor=cmap(vij),
                        **kwargs,
                    )
                )
            else:
                ax.add_patch(
                    mpl.patches.Rectangle(
                        (x + dx * s * 1.5 / L, y - dy * s),
                        s * 1.5 / L,
                        -s,
                        facecolor="#FFF",
                        edgecolor="#333",
                        **kwargs,
                    )
                )
            if draw_i is None or dy in draw_i:
                ax.text(
                    x + dx * s * 1.5 / L + s * 1.5 / L / 2,
                    y - s / 2 - dy * s,
                    "{:.2}".format(vij),
                    verticalalignment="center",
                    horizontalalignment="center",
                    fontsize=8,
                )


def draw(
    nodes,
    node_features,
    adj,
    aug_adj,
    pos,
    ax,
    highlight=None,
    key=False,
    labels=None,
    mask=None,
    draw_vector_mask=None,
    color_vector_mask=None,
    draw_nodes=None,
):
    G = nx.Graph()
    # now we add all edges to close (using pos) atoms to emphasize spatial locality
    for i in range(adj.shape[0]):
        for j in range(adj.shape[0]):
            if i != j and np.any(aug_adj):
                G.add_edge(i, j)
    for i in range(adj.shape[0]):
        for j in range(adj.shape[0]):
            if np.any(adj[i, j]):
                G.add_edge(i, j, bond=True)
    if mask is None:
        mask = [True] * len(G)
    if draw_nodes is None:
        draw_nodes = nodes
    # go from atomic number to element
    elements = np.argmax(draw_nodes, axis=-1)
    el_labels = {i: list(my_elements.values())[e] for i, e in enumerate(elements)}
    pos = nx.rescale_layout_dict(pos)
    c = ["white"] * len(G)
    all_h = []
    if highlight is not None:
        for i, h in enumerate(highlight):
            for hj in h:
                c[hj] = "C{}".format(i + 1)
                all_h.append(hj)
    # set-up edge colors based on if they are space or not
    edge_colors = ["#00000033"] * len(G.edges)
    for i, (u, v, d) in enumerate(G.edges(data=True)):
        if d.get("bond"):
            edge_colors[i] = "#000"
    nx.draw(
        G,
        ax=ax,
        pos=pos,
        labels=el_labels,
        node_size=700,
        node_color=c,
        edge_color=edge_colors,
    )
    cmap = plt.get_cmap("Wistia")
    for i in range(len(G)):
        if not mask[i]:
            continue
        if i in all_h:
            draw_vector(
                *pos[i],
                0.25,
                node_features[i],
                color_vector_mask,
                draw_vector_mask,
                ax,
                cmap,
            )
        else:
            draw_vector(
                *pos[i], 0.25, node_features[i], color_vector_mask, None, ax, None
            )
    ax.set_xlim(-1.2, 1.2)
    ax.set_ylim(-1.2, 1.2)
    ax.set_facecolor("#f5f4e9")


nodes, adj, aug_adj, pos = smiles2graph("CO")
# create plausible starting vectors
node_vectors = [[[n[1] @ [-1, 0.5, 0.3]]] for n in enumerate(nodes)]
for i in pos:
    p = pos[i]
    node_vectors[i].append(p + [p[0] - p[1]])
fig = plt.figure(figsize=(8, 5))
draw(
    nodes,
    node_vectors,
    adj,
    aug_adj,
    pos,
    plt.gca(),
    highlight=[[1], [5, 0]],
    labels=["center", "neighbors"],
)
fig.set_facecolor("#f5f4e9")
glue("dframe", plt.gcf(), display=False)

In [ ]:
# THIS CELL IS USED TO GENERATE A FIGURE
# AND NOT RELATED TO CHAPTER
# YOU CAN SKIP IT
fig, axs = plt.subplots(1, 2, squeeze=True, figsize=(14, 7.5), dpi=100)
order = [5, 1, 0, 2, 3, 4]
time_per_node = 8
last_layer = [0]
layers = 2
input_nodes = np.copy(nodes)

fig.set_facecolor("#f5f4e9")


def make_frame(t):
    axs[0].clear()
    axs[1].clear()
    layer_i = int(t / (time_per_node * len(order)))

    # I think I need to loop over k/l, not node
    out_node_vectors = np.zeros_like(node_vectors)
    for i in range(len(node_vectors[0])):
        M = len(node_vectors[0][i])
        vin = np.array([n[i] + [0] * (M - i - 2) for n in node_vectors])
        vout = np.einsum(
            "i,ij,jk->ik",
            1 / (np.sum(aug_adj, axis=1) + 1),
            aug_adj + np.eye(*aug_adj.shape),
            vin,
        )
        for j, vj in enumerate(out_node_vectors):
            vj[i] = vout[j]
    if last_layer[0] != layer_i:
        print("recomputing")
        for nv, onv in zip(node_vectors, out_node_vectors):
            onv[:] = nv
        last_layer[0] = layer_i

    t -= layer_i * time_per_node * len(order)
    i = order[int(t / time_per_node)]
    mask = [False] * nodes.shape[0]
    for j in order[: int(t / time_per_node) + 1]:
        mask[j] = True
    neighs = list(np.where(aug_adj[i])[0])
    # fraction of how far along we are
    dt = (t - int(t / time_per_node) * time_per_node) / time_per_node
    color_vector_mask = list(range(0, max(0, int((dt - 0.25) / 0.25))))
    stage = ""
    if color_vector_mask:
        stage = f"Irreps Fragments {color_vector_mask[-1]}"
    if dt >= 0.25:
        if not stage:
            stage = "Find Neighbors"
        draw(
            nodes,
            node_vectors,
            adj,
            aug_adj,
            pos,
            axs[0],
            highlight=[[i], neighs],
            color_vector_mask=color_vector_mask,
            labels=["center", "neighbors"],
            draw_nodes=input_nodes,
        )
    else:
        draw(
            nodes,
            node_vectors,
            adj,
            aug_adj,
            pos,
            axs[0],
            highlight=[[i]],
            labels=["center", "neighbors"],
            draw_nodes=input_nodes,
        )
    if dt < 0.5:
        mask[j] = False
    draw(
        nodes,
        out_node_vectors,
        adj,
        aug_adj,
        pos,
        axs[1],
        highlight=[[i]],
        mask=mask,
        color_vector_mask=color_vector_mask,
        draw_vector_mask=color_vector_mask,
        draw_nodes=input_nodes,
    )
    axs[0].set_title(f"Layer {layer_i + 1} Input\n{stage}")
    axs[1].set_title(f"Layer {layer_i + 1} Output\n{stage}")
    fig.set_facecolor("#f5f4e9")
    if t == 0:
        plt.tight_layout()
    return mplfig_to_npimage(fig)


animation = VideoClip(make_frame, duration=time_per_node * nodes.shape[0] * layers)
animation.write_gif("../_static/images/molnet.gif", fps=1)

## Cited References

```{bibliography}
:style: unsrtalpha
:filter: docname in docnames
```